In [1]:
# step 4_1 create headers = ['bc','CellClusterID','ATACFragments']
# barcodes_atach_fragments

In [2]:
%run tsv_files_utils_from_analysis.ipynb

In [3]:
# !ls ./clusters/*/*_ClusterMetadata.tsv

In [4]:
import pandas as pd

import os
import pandas as pd
clusters_folder_path = "./clusters/"



tagAlign_clusterMetadata_tsv_list = !ls ./clusters/*/*_ClusterMetadata.tsv
tagAlign_clusterMetadata_tsv_list = list(tagAlign_clusterMetadata_tsv_list)
tagAlign_clusterMetadata_tsv_list =  [x for x in tagAlign_clusterMetadata_tsv_list if "backup" not in x]
tagAlign_clusterMetadata_tsv_list

['./clusters/ENCSR023FME/ENCSR023FME_ClusterMetadata.tsv',
 './clusters/ENCSR052DKH/ENCSR052DKH_ClusterMetadata.tsv',
 './clusters/ENCSR181XXQ/ENCSR181XXQ_ClusterMetadata.tsv',
 './clusters/ENCSR205TUH/ENCSR205TUH_ClusterMetadata.tsv',
 './clusters/ENCSR261TQO/ENCSR261TQO_ClusterMetadata.tsv',
 './clusters/ENCSR302CMX/ENCSR302CMX_ClusterMetadata.tsv',
 './clusters/ENCSR367GKP/ENCSR367GKP_ClusterMetadata.tsv',
 './clusters/ENCSR397LYX/ENCSR397LYX_ClusterMetadata.tsv',
 './clusters/ENCSR411MLR/ENCSR411MLR_ClusterMetadata.tsv',
 './clusters/ENCSR453TVZ/ENCSR453TVZ_ClusterMetadata.tsv',
 './clusters/ENCSR455CVZ/ENCSR455CVZ_ClusterMetadata.tsv',
 './clusters/ENCSR474TGL/ENCSR474TGL_ClusterMetadata.tsv',
 './clusters/ENCSR506YMX/ENCSR506YMX_ClusterMetadata.tsv',
 './clusters/ENCSR538FPK/ENCSR538FPK_ClusterMetadata.tsv',
 './clusters/ENCSR618WVK/ENCSR618WVK_ClusterMetadata.tsv',
 './clusters/ENCSR660NEE/ENCSR660NEE_ClusterMetadata.tsv',
 './clusters/ENCSR718WGE/ENCSR718WGE_ClusterMetadata.tsv

In [5]:
# tagAlign_clusterMetadata_tsv_list = ["./clusters/ENCSR618WVK/ENCSR618WVK_ClusterMetadata.tsv"]
# tagAlign_clusterMetadata_tsv_list

In [6]:
tagAlign_clusterMetadata_tsv_list[0:1]

['./clusters/ENCSR023FME/ENCSR023FME_ClusterMetadata.tsv']

In [7]:
headers = ['bc','CellClusterID','ATACFragments']
# Create an empty DataFrame with the headers
df_stats = pd.DataFrame(columns=headers)

#debug
# for tagAlign_clusterMetadata_tsv in tagAlign_clusterMetadata_tsv_list[0:1]:
for tagAlign_clusterMetadata_tsv in tagAlign_clusterMetadata_tsv_list:
    datasetId = tagAlign_clusterMetadata_tsv.split("/")[-2]
    print("datasetId is ",datasetId)
    stats_output_file = os.path.join(os.path.dirname(tagAlign_clusterMetadata_tsv),'{}_barcodes_atac_fragments.tsv'.format(datasetId))
    print("stats_output_file is {}".format(stats_output_file))
    if os.path.exists(stats_output_file):
        print("stats_output_file {} exists".format(stats_output_file))
        continue
        
    df = read_tsv_file_no_index(tagAlign_clusterMetadata_tsv)
    print(df.head(2))

    num_rows_to_iterate = 1
    for index, row in df.iterrows():
        print(f"tagAlignFile: {row['tagAlignFile']}, CellClusterID: {row['CellClusterID']}")

        tagAlignFile = os.path.join(os.path.dirname(tagAlign_clusterMetadata_tsv),row['tagAlignFile'])
        print("tagAlignFile is ",tagAlignFile)
        df_tagAlign = pd.read_csv(tagAlignFile, delimiter='\t',names = ['chr','start','stop','cell_id','reads','strand'])
        print("df_tagAlign is ",df_tagAlign.head(2))
        print("df_tagAlign number of rows (+/-) is {}".format(df_tagAlign.shape[0]))
        # Filter out rows with "-" in the last column
        df_positive_strand = df_tagAlign[~(df_tagAlign.iloc[:, -1] == '-')]
        # Now you can work with the filtered DataFrame 'df_filtered'
        print(df_positive_strand.head(2))  # Example: display the first few rows of the filtered DataFrame
        number_of_fragments = df_positive_strand.shape[0]
        print("number_of_fragments = number of rows (+) is {}".format(number_of_fragments))

        # Group by 'cell_id' and count rows
        cell_id_counts = df_tagAlign.groupby('cell_id').size().reset_index(name='ATACFragments')
        print("cell_id_counts.head(2) is ",cell_id_counts.head(2))

        # Create a new DataFrame from the zipped data
        new_data = list(zip(cell_id_counts['cell_id'], [row['CellClusterID']] * len(cell_id_counts), cell_id_counts['ATACFragments']))
        new_df = pd.DataFrame(new_data, columns=['bc', 'CellClusterID', 'ATACFragments'])
        
        # Concatenate new_df with df_stats
        df_stats = pd.concat([df_stats, new_df], ignore_index=True)
        
        print("df_stats.head(2) is ",df_stats.head(2))

        # # #debug
        # num_rows_to_iterate -= 1
        # if num_rows_to_iterate == 0:
        #     break
    # Save the DataFrame to a TSV file for each dataset
    df_stats.to_csv(stats_output_file, sep='\t', index=False)




datasetId is  ENCSR023FME
stats_output_file is ./clusters/ENCSR023FME/ENCSR023FME_barcodes_atac_fragments.tsv
                                       tagAlignFile         CellClusterID  \
0  tagAlign_ENCSR023FME_Type_I_skeletal_myocyte.tsv  ENCSR023FME_Cluster1   
1               tagAlign_ENCSR023FME_Fibroblast.tsv  ENCSR023FME_Cluster2   

     ManualAnnotationLabel              TissueName  nCells MeanRNAUMIsPerCell  \
0  Type_I_skeletal_myocyte  gastrocnemius medialis    3528                 ()   
1               Fibroblast  gastrocnemius medialis     618                 ()   

   MeanATACFragmentsPerCell  
0                      4172  
1                      2623  
tagAlignFile: tagAlign_ENCSR023FME_Type_I_skeletal_myocyte.tsv, CellClusterID: ENCSR023FME_Cluster1
tagAlignFile is  ./clusters/ENCSR023FME/tagAlign_ENCSR023FME_Type_I_skeletal_myocyte.tsv
df_tagAlign is      chr  start   stop                             cell_id  reads strand
0  chr1  10013  10014  ENCSR023FME_CTTCTGATGATG

In [8]:
# df_stats.head(2)

In [9]:
# df_stats.tail(2)